# Coursera: Data Analysis Tools – Assignment 1
For this assignment the task was to run ANOVA test for some variables in the dataset selected in the previous course.
As my variables of interest are all categorical, I had to come up with a newly found quantitative variable, because ANOVA is used for the `C -> Q` cases where the explanatory variable is categorical and the response variable is quantitative.

My choice was to look at the relationship of cannabis use and perceived health. Hense my hypotheses:
* H<sub>0</sub>: There is no relationship between cannabis use and health perception;
* H<sub>A</sub>: There is some relationship between cannabis use and health perception.

My sample will be:
* The respondents who reported using cannabis within last 12 months at least once (N=1586)

The first step was to convert the information about cannabis use to quantitative values and form the sample.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi

NESARC = r"C:\Users\USER\Documents\Courses\Coursera\dai\datasets\nesarc\nesarc_pds.csv"

# Loading data
data = pd.read_csv(NESARC, low_memory=False)


### Processing variable reflecting recent cannabis use

CANNABIS_USE_12M = 'S3BD5Q2C'  # 'HOW OFTEN USED CANNABIS IN THE LAST 12 MONTHS'

# Convert values to numeric
data[CANNABIS_USE_12M] = pd.to_numeric(data[CANNABIS_USE_12M], errors='coerce')

# Code reference for CANNABIS_USE_12M
CANNABIS_USE_CAT_MAP = {
    1: "Every day",
    2: "Nearly every day",
    3: "3 to 4 times a week",
    4: "1 to 2 times a week",
    5: "2 to 3 times a month",
    6: "Once a month",
    7: "7 to 11 times a year",
    8: "3 to 6 times a year",
    9: "2 times a year",
    10: "Once a year",
    99: "Unknown",
}

# Get counts for CANNABIS_USE_12M
print(data[CANNABIS_USE_12M].value_counts(sort=False, dropna=False).rename(CANNABIS_USE_CAT_MAP))

NaN                     41479
3 to 6 times a year       192
Every day                 227
1 to 2 times a week       206
Nearly every day          121
2 times a year            149
Once a year               134
Unknown                    23
Once a month              165
2 to 3 times a month      175
3 to 4 times a week       117
7 to 11 times a year      105
Name: S3BD5Q2C, dtype: int64


In [2]:
# Recode meaningful NaN (to 11, N/A) and 'Unknowns' (99 to NaN)
data[CANNABIS_USE_12M] = data[CANNABIS_USE_12M].replace(np.NaN, 11).replace(99, np.NaN)
print(data[CANNABIS_USE_12M].value_counts(sort=False, dropna=False))

 11.0    41479
 8.0       192
 1.0       227
 4.0       206
 2.0       121
 9.0       149
 10.0      134
 6.0       165
 5.0       175
 3.0       117
NaN         23
 7.0       105
Name: S3BD5Q2C, dtype: int64


In [4]:
# Recode values so that they reflect approximately number of times of cannabis use per last year, store in new column

CANNABIS_USE_TIMES_MAP = {
    1: 365,
    2: 330,
    3: 182,
    4: 104,
    5: 75,
    6: 12,
    7: 9,
    8: 4.5,
    9: 2,
    10: 1,
    11: 0
}

def recode_values(row):
    return CANNABIS_USE_TIMES_MAP.get(row[CANNABIS_USE_12M])

CANNABIS_USE_QUANT = 'CANNABIS_USE_QUANT'

data[CANNABIS_USE_QUANT] = data.apply(lambda row: recode_values(row), axis=1)

print(data[CANNABIS_USE_QUANT].value_counts(sort=False, dropna=False))

 0.0      41479
 1.0        134
 2.0        149
 4.5        192
 9.0        105
 330.0      121
 75.0       175
 365.0      227
 12.0       165
NaN          23
 104.0      206
 182.0      117
Name: CANNABIS_USE_QUANT, dtype: int64


In [6]:
# Convert CANNABIS_USE_QUANT to numeric
data[CANNABIS_USE_QUANT] = pd.to_numeric(data[CANNABIS_USE_QUANT], errors='coerce')

# Make a subset only those who used cannabis within past 12 months
subset_cannabis = data[data[CANNABIS_USE_QUANT] > 0].copy()

print(subset_cannabis[CANNABIS_USE_QUANT].value_counts(sort=False, dropna=False))

4.5      192
1.0      134
2.0      149
9.0      105
104.0    206
12.0     165
330.0    121
75.0     175
182.0    117
365.0    227
Name: CANNABIS_USE_QUANT, dtype: int64


In [8]:
# Introduce health perception as categorical variable
HEALTH_PERCEPTION = 'S1Q16'  # 'SELF-PERCEIVED CURRENT HEALTH'

# Convert values to numeric
subset_cannabis[HEALTH_PERCEPTION] = pd.to_numeric(data[HEALTH_PERCEPTION], errors='coerce')

# Recode Unknowns
subset_cannabis[HEALTH_PERCEPTION] = subset_cannabis[HEALTH_PERCEPTION].replace(9, np.nan)

# Create dataframe for only 2 variables
subset_cann_health = subset_cannabis[[CANNABIS_USE_QUANT, HEALTH_PERCEPTION]].dropna()

# Use .ols method to get the F-statistic and associated p-value
formula_health_cannab = '{} ~ C({})'.format(CANNABIS_USE_QUANT, HEALTH_PERCEPTION)
model_health_cannab = smf.ols(formula=formula_health_cannab, data=subset_cann_health)
results_health_cannab = model_health_cannab.fit()
print (results_health_cannab.summary())

                            OLS Regression Results                            
Dep. Variable:     CANNABIS_USE_QUANT   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     5.699
Date:                Thu, 07 Feb 2019   Prob (F-statistic):           0.000149
Time:                        17:05:11   Log-Likelihood:                -10034.
No. Observations:                1586   AIC:                         2.008e+04
Df Residuals:                    1581   BIC:                         2.010e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          97.5121      6.668     